In [2]:
import pandas as pd
import numpy as np
import os
import healpy as hp
from astroquery.vizier import Vizier
from astroquery.gaia import Gaia
#import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.time import Time
import math
from astropy.io import ascii

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [3]:
'''Rename columns'''
def rename_columns(df, cat):
    '''First variant witjout source_id'''
    
    old_columns = df.columns[5:].tolist()
    new_columns = []
    
    for column in old_columns[:3]:
        new_columns.append(cat+column[3:])
    
    for column in old_columns[3:]:
        new_columns.append(column+'_gaia-'+cat)    
    
    for i in range(len(new_columns)):
        df = df.rename(columns= {old_columns[i]: new_columns[i]})
    
    return df

def rename_columns_id(df,cat):
    #df = df.drop(columns=['new_pmksi', 'new_pmeta', 'F'])
    df = df.rename(columns = {'cat_ra': cat+'_ra',
                   'cat_dec': cat+'_dec',
                   'dt': cat+'_dt',
                   'dm': cat+'_dm',
                   'dist_sec': cat+'_rsec',
                   'new_pmksi':cat+'_pmski',
                   'new_pmeta':cat+'_pmeta',
                   'F':cat+'_F'})
    return df
    
    

In [4]:
'''Global variables'''

radsec = 600
min_delta_mag = 10
limmag = 18

info = pd.read_csv('cat_info.csv').fillna('NaN')
info

,ra,dec,ra_error,dec_error,ref_epoch,phot_g_mean_mag,link,pecularity
CMC15,RA_ICRS,DE_ICRS,e_RA_ICRS,e_DE_ICRS,MJD-51263,r_mag,I/327/cmc15,arcsec|51263:mjd
Pan-STARRS DR1,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,_tab1_10,gmag,II/349/ps1,arcsec|0:mjd
2MASS All-Sky,RAJ2000,DEJ2000,errMaj,errMin,_tab1_36,Jmag,II/246/out,arcsec:errPA|0:jd
SDSS DR12,RA_ICRS,DE_ICRS,e_RA_ICRS,e_RA_ICRS,ObsDate,gmag,V/147/sdss12,arcsec|jy
URAT1,RAJ2000,DEJ2000,sigs,sigm,Epoch,gmag,I/329/urat1,mas|jy
UCAC4,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,EpRA,gmag,I/322A/out,mas|EpDE


In [5]:
indir = 'catalog_results'
df_urat = pd.read_csv(os.path.join(indir,'URAT1.csv'))#, index_col = 0)
df_ucac = pd.read_csv(os.path.join(indir,'UCAC4.csv'))#, index_col = 0)
#df_urat = rename_columns(df_urat, 'urat1')
#df_ucac = rename_columns(df_ucac, 'ucac4')
display(df_urat)
display(df_ucac)

,Unnamed: 0,gaia_ra,gaia_dec,cat_ra,cat_dec,dt,dm,dist_sec,gaia_pmra,gaia_pmdec,new_pmksi,new_pmeta,F
0,3999433452516434560,180.185765,20.848187,180.185833,20.848086,1.6,1.67,0.414,-129.496727,235.954725,13.216994,8.291073,65.689485
1,3999432971480096640,180.136496,20.815287,180.136574,20.815168,1.9,NaN,0.490,-130.134956,235.456935,14.118043,11.668930,72.860836
2,2068138392112597376,308.009644,41.926820,308.009628,41.926843,2.0,NaN,0.088,21.121711,-5.906501,1.963647,16.595884,6.068738
3,2288204854134519424,295.018022,73.911317,295.022253,73.914252,1.7,NaN,11.357,-23.876102,296.312861,8.956734,-25.568448,77.457928
4,2877522230262577024,2.169965,36.626590,2.170056,36.626655,2.0,NaN,0.325,-114.068699,-124.300389,-2.260762,-19.802216,30.918692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12738,678820100826341376,128.262404,25.196487,128.262283,25.196552,1.9,NaN,0.485,218.142942,-119.555329,8.305120,-3.657528,41.530842
12739,678715544143013632,127.094169,24.499971,127.094239,24.500022,1.6,1.13,0.292,-144.377017,-126.374370,4.536356,-15.928574,46.522199
12740,509442567963973376,22.727863,59.964030,22.727831,59.964177,2.2,NaN,0.537,30.235683,-221.275224,5.882612,-7.064419,53.812321
12741,4317732619843065728,297.358489,15.112969,297.365632,15.109827,1.5,4.35,27.270,-9.282367,13.084218,19.476172,-92.712088,8.629477


,Unnamed: 0,gaia_ra,gaia_dec,cat_ra,cat_dec,dt,dm,dist_sec,gaia_pmra,gaia_pmdec,new_pmksi,new_pmeta,F
0,4892429171367335040,65.400190,-27.046093,65.400243,-27.046217,38.5,2.22,0.460,-12.468865,20.545892,3.009879,-0.064844,21.117375
1,6027693584407085056,254.393003,-31.375706,254.393527,-31.378187,18.2,NaN,9.089,-12.919468,8.647653,-11.098598,-26.467105,13.026387
2,6027685097545659136,254.298860,-31.520545,254.290053,-31.519265,18.3,5.79,27.441,-5.669891,8.700257,3.544486,-14.637690,9.880359
3,6027685544222718208,254.071639,-31.628937,254.076908,-31.635925,19.3,4.33,29.894,-2.753096,3.776416,27.696487,-16.397226,13.501607
4,6027681558489999232,254.175604,-31.674378,254.176516,-31.676866,22.7,NaN,9.389,-13.301846,7.345171,-7.318569,-6.582581,6.993427
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2810429858936372480,349.742708,10.713867,349.736396,10.713882,18.4,0.65,22.354,261.895436,-48.791667,1.661342,1.977632,129.558722
1088,5368615390942233856,177.787523,-52.905409,177.794257,-52.900763,24.3,5.28,22.206,12.511030,17.865459,-25.658006,7.634994,14.296875
1089,5936491976788877824,256.076864,-51.432692,256.077871,-51.432613,22.2,1.94,2.252,-146.664122,-19.606915,26.065692,-1.990697,82.809420
1090,5829999710095756416,248.417904,-62.735532,248.416762,-62.735223,17.7,4.81,2.201,-2.480382,6.304568,-3.146264,-8.453136,7.860925


In [6]:
df_urat = rename_columns_id(df_urat, 'urat1')
df_ucac = rename_columns_id(df_ucac, 'ucac4')
display(df_urat)
display(df_ucac)

,Unnamed: 0,gaia_ra,gaia_dec,urat1_ra,urat1_dec,urat1_dt,urat1_dm,urat1_rsec,gaia_pmra,gaia_pmdec,urat1_pmski,urat1_pmeta,urat1_F
0,3999433452516434560,180.185765,20.848187,180.185833,20.848086,1.6,1.67,0.414,-129.496727,235.954725,13.216994,8.291073,65.689485
1,3999432971480096640,180.136496,20.815287,180.136574,20.815168,1.9,NaN,0.490,-130.134956,235.456935,14.118043,11.668930,72.860836
2,2068138392112597376,308.009644,41.926820,308.009628,41.926843,2.0,NaN,0.088,21.121711,-5.906501,1.963647,16.595884,6.068738
3,2288204854134519424,295.018022,73.911317,295.022253,73.914252,1.7,NaN,11.357,-23.876102,296.312861,8.956734,-25.568448,77.457928
4,2877522230262577024,2.169965,36.626590,2.170056,36.626655,2.0,NaN,0.325,-114.068699,-124.300389,-2.260762,-19.802216,30.918692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12738,678820100826341376,128.262404,25.196487,128.262283,25.196552,1.9,NaN,0.485,218.142942,-119.555329,8.305120,-3.657528,41.530842
12739,678715544143013632,127.094169,24.499971,127.094239,24.500022,1.6,1.13,0.292,-144.377017,-126.374370,4.536356,-15.928574,46.522199
12740,509442567963973376,22.727863,59.964030,22.727831,59.964177,2.2,NaN,0.537,30.235683,-221.275224,5.882612,-7.064419,53.812321
12741,4317732619843065728,297.358489,15.112969,297.365632,15.109827,1.5,4.35,27.270,-9.282367,13.084218,19.476172,-92.712088,8.629477


,Unnamed: 0,gaia_ra,gaia_dec,ucac4_ra,ucac4_dec,ucac4_dt,ucac4_dm,ucac4_rsec,gaia_pmra,gaia_pmdec,ucac4_pmski,ucac4_pmeta,ucac4_F
0,4892429171367335040,65.400190,-27.046093,65.400243,-27.046217,38.5,2.22,0.460,-12.468865,20.545892,3.009879,-0.064844,21.117375
1,6027693584407085056,254.393003,-31.375706,254.393527,-31.378187,18.2,NaN,9.089,-12.919468,8.647653,-11.098598,-26.467105,13.026387
2,6027685097545659136,254.298860,-31.520545,254.290053,-31.519265,18.3,5.79,27.441,-5.669891,8.700257,3.544486,-14.637690,9.880359
3,6027685544222718208,254.071639,-31.628937,254.076908,-31.635925,19.3,4.33,29.894,-2.753096,3.776416,27.696487,-16.397226,13.501607
4,6027681558489999232,254.175604,-31.674378,254.176516,-31.676866,22.7,NaN,9.389,-13.301846,7.345171,-7.318569,-6.582581,6.993427
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2810429858936372480,349.742708,10.713867,349.736396,10.713882,18.4,0.65,22.354,261.895436,-48.791667,1.661342,1.977632,129.558722
1088,5368615390942233856,177.787523,-52.905409,177.794257,-52.900763,24.3,5.28,22.206,12.511030,17.865459,-25.658006,7.634994,14.296875
1089,5936491976788877824,256.076864,-51.432692,256.077871,-51.432613,22.2,1.94,2.252,-146.664122,-19.606915,26.065692,-1.990697,82.809420
1090,5829999710095756416,248.417904,-62.735532,248.416762,-62.735223,17.7,4.81,2.201,-2.480382,6.304568,-3.146264,-8.453136,7.860925


In [17]:
df = df_urat.merge(df_ucac)#.set_index('Unnamed: 0')
df['source_id'] = df['Unnamed: 0']
df = df.drop(columns = ['Unnamed: 0'])
df.set_index('source_id', inplace = True)
df['dt'] = df.ucac4_dt - df.urat1_dt
df[df.gaia_ra<1]

,gaia_ra,gaia_dec,urat1_ra,urat1_dec,urat1_dt,urat1_dm,urat1_rsec,gaia_pmra,gaia_pmdec,urat1_pmski,...,urat1_F,ucac4_ra,ucac4_dec,ucac4_dt,ucac4_dm,ucac4_rsec,ucac4_pmski,ucac4_pmeta,ucac4_F,dt
source_id,,,,,,,,,,,,,,,,,,,,,
431635455820288128,0.603372,63.962728,0.621836,63.961976,2.0,1.01,29.276,918.852510,108.269167,-12.964823,...,84.001018,0.621862,63.962018,12.7,1.01,29.303,-2.393732,-1.501401,369.242400,10.7
2772447981373673344,0.196000,16.402230,0.195991,16.402295,2.0,1.68,0.232,12.027882,-119.762952,17.850653,...,21.671689,0.195898,16.402799,16.8,1.70,2.074,2.330447,-0.365441,50.992008,14.8


In [63]:
'''Начало основного кода.'''

'''У меня уже есть табличка с отождественными близкими звездами gaia в разных кталогах.
Читаю табличку с инфой про каталоги, выбираю данные гайя (0-3) и нужные каталоги (8 - URAT1,9 - UCAC4),
удаляю строчки, для которых нет информации по какому-то какалогу.'''

df = pd.read_csv('catalogue_positions.csv').iloc[:, [
    0, 1, 2, 3, 8, 9]].dropna()

# df.info()

In [64]:
df[df.ra<1]

,ra,dec,pmra,pmdec,URAT1[dm|r|ra|dec|t],UCAC4[dm|r|ra|dec|t]
70,0.603372,63.962728,918.852510,108.269167,1.01|29.276|0.6218361|63.9619758|2013.5,1.01|29.303|0.6218621|63.9620181|2002.8
738,0.196000,16.402230,12.027882,-119.762952,1.68|0.232|0.1959906|16.4022947|2013.5,1.7|2.074|0.1958983|16.4027987|1998.7
2912,0.726463,46.234038,192.346503,-22.843691,1.54|26.46|0.7169439|46.2307914|2013.5,1.54|26.41|0.7169842|46.2307664|1995.3
2913,0.330265,46.437554,-21.438372,-9.628835,3.85|26.864|0.3234617|46.4433553|2013.5,4.8|16.105|0.3242606|46.4392364|2001.5
3136,0.189270,55.049948,-67.946252,-163.408952,nan|0.371|0.1893286|55.05005|2013.5,nan|2.097|0.1896974|55.0504823|2000.0
4213,0.752043,16.929457,117.031746,33.080639,1.12|0.255|0.7519811|16.9294353|2013.5,1.13|1.903|0.7515218|16.9293064|1999.1
4214,0.200197,16.986791,-52.696037,-307.113657,1.19|0.582|0.2002317|16.9869536|2013.7,1.19|4.935|0.2003962|16.9881523|1991.3
4696,0.223191,35.752445,158.790664,-41.084024,1.02|0.343|0.2230861|35.7524672|2013.5,1.07|2.567|0.2223486|35.75262|1991.4
6793,0.780577,6.275371,239.476753,-512.968429,2.13|1.528|0.7803892|6.2757511|2012.9,2.13|8.362|0.7795762|6.2774689|2000.0
8973,0.111111,50.320749,-27.104467,20.681041,5.45|2.664|0.1107869|50.3200436|2013.5,5.45|2.692|0.1107756|50.3200378|2001.2


In [65]:
'''Нужно разобрать по столбцам блоки с информацией каталогов.
Поскольку в данном случае urat - более новый каталог, чем ucac, то:
ucac4: все данные будут с индексом 0
urat1: все данные будут с индексом 1
Для расчетов нужно из данных с индексом 1 вычитать данные с индексом 0.'''

cats = ['UCAC4', 'URAT1']
cols = ['dm', 'r', 'ra', 'dec', 't']

for i, col in enumerate(cols):
    for j, cat in enumerate(cats):
        block = cat+'[dm|r|ra|dec|t]'
        df[col+str(j)] = df[block]
        df[col+str(j)] = df[col+str(j)].apply(lambda x: x.split('|')[i])
for cat in cats:
    df.drop(columns=[cat+'[dm|r|ra|dec|t]'], inplace=True)
df

,ra,dec,pmra,pmdec,dm0,dm1,r0,r1,ra0,ra1,dec0,dec1,t0,t1
13,180.185765,20.848187,-129.496727,235.954725,1.72,1.67,3.779,0.414,180.1863359,180.1858329,20.8472778,20.8480858,2000.0,2013.9
26,2.169965,36.626590,-114.068699,-124.300389,nan,nan,2.587,0.325,2.1705765,2.1700561,36.6271245,36.626655,1991.2,2013.5
29,322.998168,-1.335470,-95.491442,-239.377622,1.8,1.83,3.811,0.576,322.9985636,322.9982392,-1.3344803,-1.3353186,1998.1,2013.0
35,20.870725,15.292488,-33.315634,-312.292719,1.19,1.26,17.771,17.753,20.8693183,20.8694047,15.2877445,15.2877264,1999.4,2013.3
36,9.889655,60.553080,-157.509438,-292.543034,nan,nan,5.042,0.679,9.8909739,9.8898264,60.5543273,60.5532542,2003.6,2013.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12759,293.612710,3.923355,-58.723485,38.797843,0.22,0.28,22.509,22.528,293.6064553,293.60645,3.9230767,3.9230789,2000.5,2014.0
12760,53.611355,67.833935,30.210234,-24.623834,6.53,6.51,21.51,21.087,53.5955615,53.5958669,67.8342559,67.8342097,2000.2,2013.5
12761,3.860113,60.539471,31.490063,-184.236992,4.92,4.27,26.006,28.641,3.8594792,3.8749835,60.5322562,60.5363233,1999.2,2013.5
12762,167.468642,21.625418,52.412829,-21.791442,1.24,1.24,0.919,0.124,167.468395,167.4686131,21.6255159,21.6254283,1991.2,2014.0


In [66]:
'''Для каждой звезды нужно рассчитать новое собственное движение с разностью по каталогам. '''

'''0. Выберем любую звезду для отладки кода.'''

star = df.iloc[13]

'''1. Назначим константы.'''
# диаметр области каталога, из которой формируется набор опорных звезд
radsec = 30
# предельная минимальная звездная величина (ярче не пройдут)
start_delta_mag = 10

'''2. Чтобы сделать запрос для получения опорных звезд, 
нужно взять информацию о каждом каталоге из файла cats_info.csv'''

info = pd.read_csv('cat_info.csv')  # таблица с данными всех каталогов
# info
cat0 = info.loc['UCAC4']  # данные для запроса каталога ucac4
cat1 = info.loc['URAT1']  # данные для запроса каталога urat1
# cat0, cat1
# star

In [ ]:
'''Скачиваем датафреймы с опорными звездами: df0 для ucac4 и df1 для urat1.'''

def CatTable(star, catalog):
    global info
    global radsec

    try:
        link = info.loc[catalog]['link']
        viz = Vizier(columns=['all'])
        table = viz.query_region(SkyCoord(
            ra=star.cat_ra, dec=star.cat_dec, unit=(u.deg, u.deg), frame='icrs'),
            radius=radsec * u.arcsec, catalog=link)[0]

        df = pd.DataFrame.from_records(table.as_array())

        pecularities = info.pecularity[catalog]
        pecs = pecularities.split('|')

        if (not pecs[0] == 'mas'):

            column1 = info['ra_error'][catalog]
            column2 = info['dec_error'][catalog]

            df[column1] = df[column1].apply(lambda x: x * 1000)
            df[column2] = df[column2].apply(lambda x: x * 1000)

            if len(pecs[0].split(':')) == 2:
                df[['errMaj', 'errMin', 'errPA']] = \
                    df[['errMaj', 'errMin', 'errPA']].apply(errors_2MASS, axis=1)

        if (not pecs[1] == 'jy'):
            ps = pecs[1].split(':')

            if len(ps) == 2:
                column = info['ref_epoch'][catalog]

                df[column] = \
                    df[column].apply(lambda x: Time((x + int(ps[0])), format=ps[1]).jyear)

            else:
                df[['EpRA', 'EpDE']] = \
                    df[['EpRA', 'EpDE']].apply(epoch_UCAC4, axis=1)

        cols = info.loc[catalog].tolist()[:-2]
        df = df[['_r'] + cols]
        df = RenameColumns(df, catalog)
        df._r = round(df._r, 4)
        df.ref_epoch = round(df.ref_epoch, 1)
        df = df.sort_values(by=['_r'])
        #print(catalog)  # + ': success')

        return [df, True]

    except Exception as e:
        print(catalog + '\t' + ' ERROR:', e)
        return ['NaN', False]

In [ ]:
'''Приводим датафреймы с единой форме, 
учитываем обосенности ошибок положений и эпохи наблюдений (столбец pecularity).'''



In [ ]:
'''Формируем GydeStarsDF'''

def GudeTable(gaiadf, catdf):
    gudedf = pd.DataFrame(columns=['k_g', 'e_g', 'k_c', 'e_c'])
    tg = 2015.5
    median_epoch = catdf.describe().ref_epoch['50%']
    for gaia_id in list(gaiadf.index):
        gstar = gaiadf.loc[gaia_id]
        pmr = gstar.pmra
        pmd = gstar.pmdec
        bias = 10000
        found = False
        for cat_id in list(catdf.index):
            cstar = catdf.loc[cat_id]
            t = cstar.ref_epoch
            k_c, e_c = tangFromRADE(cstar.ra * math.pi / 180., cstar.dec * math.pi / 180.,
                                    star.gaia_ra * math.pi / 180., star.gaia_dec * math.pi / 180.)  # [rad]
            # to take into account pm
            alpha, delta = RADecFromTang((t - tg) * math.pi * pmr / 648000000.0,
                                         (t - tg) * math.pi * pmd / 648000000.0,
                                         math.radians(gstar.ra), math.radians(gstar.dec))
            # rr, dd = math.degrees(alpha), math.degrees(delta)
            k_g, e_g = tangFromRADE(alpha, delta, star.gaia_ra * math.pi / 180.0,
                                    star.gaia_dec * math.pi / 180.0)  # [radians]
            biasK = abs(k_c - k_g) * 648000000.0 / math.pi  # [mas]
            biasE = abs(e_c - e_g) * 648000000.0 / math.pi  # [mas]
            current_bias = math.sqrt(biasE ** 2 + biasK ** 2)
            if current_bias < bias:
                bias = current_bias
                found = True
                drop_id = cat_id
                gude_star = [k_g * 648000000.0 / math.pi, e_g * 648000000.0 / math.pi,
                             k_c * 648000000.0 / math.pi, e_c * 648000000.0 / math.pi]
        if found:
            gudedf.loc[gaia_id] = gude_star
            catdf = catdf.drop(index=drop_id)
    return gudedf

In [84]:
'''
df_urat = pd.read_csv('URAT1.csv')
df_ucac = pd.read_csv('UCAC4.csv')
df_urat = rename_columns(df_urat, 'urat1')
df_ucac = rename_columns(df_ucac, 'ucac4')
display(df_urat)
display(df_ucac)

df = df_urat.merge(df_ucac)

ra = df.gaia_ra.tolist()
dec = df.gaia_dec.tolist()
indexes = []
for i in range(len(ra)):
    job = Gaia.launch_job_async("SELECT * FROM gaiadr2.gaia_source \
    WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec), \
    CIRCLE('ICRS',%f,%f,%f))=1 AND  phot_g_mean_mag<%f AND pmra IS NOT NULL AND abs(pmra)>0 AND pmdec IS NOT NULL AND abs(pmdec)>0;" \
                            % (ra[i], dec[i], 0.0001, 18) , dump_to_file=False)
    gaiat = job.get_results()
    indexes.append(gaiat['source_id'][0])
    
df['gaia_index'] = indexes
df.set_index('gaia_index')

'''

'\ndf_urat = pd.read_csv(\'URAT1.csv\')\ndf_ucac = pd.read_csv(\'UCAC4.csv\')\ndf_urat = rename_columns(df_urat, \'urat1\')\ndf_ucac = rename_columns(df_ucac, \'ucac4\')\ndisplay(df_urat)\ndisplay(df_ucac)\n\ndf = df_urat.merge(df_ucac)\n\nra = df.gaia_ra.tolist()\ndec = df.gaia_dec.tolist()\nindexes = []\nfor i in range(len(ra)):\n    job = Gaia.launch_job_async("SELECT * FROM gaiadr2.gaia_source     WHERE CONTAINS(POINT(\'ICRS\',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec),     CIRCLE(\'ICRS\',%f,%f,%f))=1 AND  phot_g_mean_mag<%f AND pmra IS NOT NULL AND abs(pmra)>0 AND pmdec IS NOT NULL AND abs(pmdec)>0;"                             % (ra[i], dec[i], 0.0001, 18) , dump_to_file=False)\n    gaiat = job.get_results()\n    indexes.append(gaiat[\'source_id\'][0])\n    \ndf[\'gaia_index\'] = indexes\ndf.set_index(\'gaia_index\')\n\n'

In [67]:
pd.read_csv('catalogue_positions.csv')

,ra,dec,pmra,pmdec,CMC15[dm|r|ra|dec|t],Pan-STARRS DR1[dm|r|ra|dec|t],2MASS All-Sky[dm|r|ra|dec|t],SDSS DR12[dm|r|ra|dec|t],URAT1[dm|r|ra|dec|t],UCAC4[dm|r|ra|dec|t]
0,65.400190,-27.046093,-12.468865,20.545892,0.73|0.232|65.400237|-27.046149|2009.3,2.09|0.009|65.40019667|-27.04609621|2012.7,3.15|0.413|65.400263|-27.046194|1999.7,NaN,NaN,2.22|0.46|65.4002427|-27.0462175|1977.0
1,254.393003,-31.375706,-12.919468,8.647653,2.74|7.952|254.39371|-31.377828|2008.4,0.01|29.971|254.38793868|-31.36859129|2014.1,3.65|20.334|254.39934|-31.374056|1998.5,NaN,NaN,nan|9.089|254.3935274|-31.3781873|1997.3
2,254.298860,-31.520545,-5.669891,8.700257,2.66|29.094|254.29408|-31.527516|2011.2,0.05|16.279|254.29682008|-31.52471202|2014.4,3.61|19.447|254.305199|-31.520784|1998.5,NaN,NaN,5.79|27.441|254.2900527|-31.5192653|1997.2
3,254.071639,-31.628937,-2.753096,3.776416,2.46|27.135|254.070567|-31.636414|2005.5,0.05|16.494|254.07154791|-31.63351384|2014.4,3.57|23.066|254.076911|-31.633512|1998.5,NaN,NaN,4.33|29.894|254.0769083|-31.6359253|1996.2
4,254.175604,-31.674378,-13.301846,7.345171,2.7|15.832|254.175861|-31.669981|2008.4,0.05|18.383|254.17978294|-31.67070209|2014.4,3.81|27.89|254.178304|-31.666973|1998.5,NaN,NaN,nan|9.389|254.1765159|-31.6768662|1992.8
...,...,...,...,...,...,...,...,...,...,...
12766,116.861636,-16.758382,-2.525590,12.575356,NaN,0.35|15.321|116.86593431|-16.75949528|2011.7,4.02|25.622|116.854282|-16.759377|1998.2,NaN,NaN,NaN
12767,116.939309,-16.706801,7.549478,5.381499,3.0|29.047|116.931245|-16.704492|2005.5,0.07|19.294|116.93386433|-16.70559572|2012.0,2.78|19.939|116.941562|-16.701704|1998.2,NaN,NaN,2.46|19.071|116.9437903|-16.7036945|1999.1
12768,116.733773,-16.890728,14.950998,0.775526,3.82|27.726|116.731797|-16.88327|2005.3,0.13|21.18|116.72840547|-16.89359277|2011.8,2.8|21.231|116.7284|-16.893612|1998.2,NaN,NaN,3.44|27.555|116.7317774|-16.8833239|1990.6
12769,211.453175,-60.549052,22.632929,56.925836,NaN,NaN,2.1|24.713|211.439763|-60.55093|2000.1,NaN,NaN,7.95|6.709|211.4568274|-60.5495803|1994.2
